In [64]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link><main>
<article class="day-desc read-aloud"><h2>--- Day 25: The Halting Problem ---</h2><p>Following the twisty passageways deeper and deeper into the CPU, you finally reach the <span title="Get it? CPU core?">core</span> of the computer. Here, in the expansive central chamber, you find a grand apparatus that fills the entire room, suspended nanometers above your head.</p>
<p>You had always imagined CPUs to be noisy, chaotic places, bustling with activity. Instead, the room is quiet, motionless, and dark.</p>
<p>Suddenly, you and the CPU's <em>garbage collector</em> startle each other. "It's not often we get  many visitors here!", he says. You inquire about the stopped machinery.</p>
<p>"It stopped milliseconds ago; not sure why. I'm a garbage collector, not a doctor." You ask what the machine is for.</p>
<p>"Programs these days, don't know their origins. That's the <em>Turing machine</em>! It's what makes the whole computer work." You try to explain that Turing machines are merely models of computation, but he cuts you off. "No, see, that's just what they <em>want</em> you to think. Ultimately, inside every CPU, there's a Turing machine driving the whole thing! Too bad this one's broken. <a href="https://www.youtube.com/watch?v=cTwZZz0HV8I">We're doomed!</a>"</p>
<p>You ask how you can help. "Well, unfortunately, the only way to get the computer running again would be to create a whole new Turing machine from scratch, but there's no <em>way</em> you can-" He notices the look on your face, gives you a curious glance, shrugs, and goes back to sweeping the floor.</p>
<p>You find the <em>Turing machine blueprints</em> (your puzzle input) on a tablet in a nearby pile of debris. Looking back up at the broken Turing machine above, you can start to identify its parts:</p>
<ul>
<li>A <em>tape</em> which contains <code>0</code> repeated infinitely to the left and right.</li>
<li>A <em>cursor</em>, which can move left or right along the tape and read or write values at its current position.</li>
<li>A set of <em>states</em>, each containing rules about what to do based on the current value under the cursor.</li>
</ul>
<p>Each slot on the tape has two possible values: <code>0</code> (the starting value for all slots) and <code>1</code>. Based on whether the cursor is pointing at a <code>0</code> or a <code>1</code>, the current state says <em>what value to write</em> at the current position of the cursor, whether to <em>move the cursor</em> left or right one slot, and <em>which state to use next</em>.</p>
<p>For example, suppose you found the following blueprint:</p>
<pre><code>Begin in state A.
Perform a diagnostic checksum after 6 steps.

In state A:
If the current value is 0: - Write the value 1. - Move one slot to the right. - Continue with state B.
If the current value is 1: - Write the value 0. - Move one slot to the left. - Continue with state B.

In state B:
If the current value is 0: - Write the value 1. - Move one slot to the left. - Continue with state A.
If the current value is 1: - Write the value 1. - Move one slot to the right. - Continue with state A.
</code></pre>

<p>Running it until the number of steps required to take the listed <em>diagnostic checksum</em> would result in the following tape configurations (with the <em>cursor</em> marked in square brackets):</p>
<pre><code>... 0  0  0 [0] 0  0 ... (before any steps; about to run state A)
... 0  0  0  1 [0] 0 ... (after 1 step;     about to run state B)
... 0  0  0 [1] 1  0 ... (after 2 steps;    about to run state A)
... 0  0 [0] 0  1  0 ... (after 3 steps;    about to run state B)
... 0 [0] 1  0  1  0 ... (after 4 steps;    about to run state A)
... 0  1 [1] 0  1  0 ... (after 5 steps;    about to run state B)
... 0  1  1 [0] 1  0 ... (after 6 steps;    about to run state A)
</code></pre>
<p>The CPU can confirm that the Turing machine is working by taking a <em>diagnostic checksum</em> after a specific number of steps (given in the blueprint). Once the specified number of steps have been executed, the Turing machine should pause; once it does, count the number of times <code>1</code> appears on the tape. In the above example, the <em>diagnostic checksum</em> is <em><code>3</code></em>.</p>
<p>Recreate the Turing machine and save the computer! <em>What is the diagnostic checksum</em> it produces once it's working again?</p>
</article>


In [65]:
from pprint import pprint
from re import findall, split

from more_itertools import last
from tabulate import tabulate


example = """
Begin in state A.
Perform a diagnostic checksum after 6 steps.

In state A:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state B.
  If the current value is 1:
    - Write the value 0.
    - Move one slot to the left.
    - Continue with state B.

In state B:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the left.
    - Continue with state A.
  If the current value is 1:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state A.
"""


class TuringMachine:
    def __init__(self, blueprint: str) -> None:
        init_block, *states = split(r"\n\s*\n", blueprint.strip())
        l1, l2 = init_block.splitlines()
        self.state = last(findall(r"[A-Z]{1}", l1))
        self.steps = int(last(findall(r"\d+", l2)))
        self.tape = defaultdict(int)
        self.cursor = 0

        self.states = {}
        for state in states:
            st, _, w0, m0, c0, _, w1, m1, c1 = state.splitlines()
            st = last(findall(r"[A-Z]{1}", st))

            w0 = int(last(findall(r"\d+", w0)))
            m0 = 1 if "right" in m0 else -1
            c0 = last(findall(r"[A-Z]{1}", c0))

            w1 = int(last(findall(r"\d+", w1)))
            m1 = 1 if "right" in m1 else -1
            c1 = last(findall(r"[A-Z]{1}", c1))

            self.states[st] = w0, m0, c0, w1, m1, c1

    def diagnostic_checksum(self) -> int:
        checksum = 0
        for _ in range(self.steps):
            w0, m0, c0, w1, m1, c1 = self.states[self.state]
            if self.tape[self.cursor] == 0:
                if self.tape[self.cursor] == 0 and w0 == 1:
                    self.tape[self.cursor] = w0
                    checksum += 1
                elif self.tape[self.cursor] == 1 and w0 == 0:
                    self.tape[self.cursor] = w0
                    checksum -= 1

                self.cursor += m0
                self.state = c0
            else:
                if self.tape[self.cursor] == 0 and w1 == 1:
                    self.tape[self.cursor] = w1
                    checksum += 1
                elif self.tape[self.cursor] == 1 and w1 == 0:
                    self.tape[self.cursor] = w1
                    checksum -= 1

                self.cursor += m1
                self.state = c1

        return checksum

    def __repr__(self) -> str:
        r = [f"TuringMachine:", f"\t{self.state=}", f"\t{self.steps=}", "\tStates:"]
        for st, (w0, m0, c0, w1, m1, c1) in self.states.items():
            r += [f"\t\tIn state {st}."]
            r += [f"\t\t\tIf current value is 0."]
            r += [f"\t\t\t\t- Write the value {w0}."]
            r += [f"\t\t\t\t- Move one slot to the {'right' if m0 else 'left'}."]
            r += [f"\t\t\t\t- Continue with state {c0}."]
            r += [f"\t\t\tIf current value is 1."]
            r += [f"\t\t\t\t- Write the value {w1}."]
            r += [f"\t\t\t\t- Move one slot to the {'right' if m1 else 'left'}."]
            r += [f"\t\t\t\t- Continue with state {c1}."]
        return "\n".join(r)


print(f"Example: {TuringMachine(example).diagnostic_checksum()} should be 3.")

Example: 3 should be 3.


In [66]:
with open("../input/day25.txt") as f:
    puzzle = f.read()

print(f"Part I: {TuringMachine(puzzle).diagnostic_checksum()}")

Part I: 4769


<link href="style.css" rel="stylesheet"></link><main>
<main>

<p>Your puzzle answer was <code>4769</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The Turing machine, and soon the entire computer, springs back to life.  A console glows dimly nearby, awaiting your command.</p>
<pre><code>&gt; reboot printer
Error: That command requires <em>priority 50</em>. You currently have <em>priority 0</em>.
You must deposit <em class="star">50 stars</em> to increase your priority to the required level.
</code></pre>
<p>The console flickers for a moment, and then prints another message:</p>
<pre><code><em class="star">Star</em> accepted.
You must deposit <em class="star">49 stars</em> to increase your priority to the required level.
</code></pre>
<p>The <em>garbage collector</em> winks at you, then continues sweeping.</p>
</article>

</main>


<link href="style.css" rel="stylesheet"></link><main>
<main>
<article>
<p>You deposit all <span class="day-success">fifty stars</span> and reboot the printer. Suddenly, everything seems a lot less pixelated than before.</p><p>"--raise your priority level enough to send the reboot command and... hey look, it's printing! I'll bring it to Santa. Thanks!" She runs off.</p>
<p class="aside">Congratulations!  You've finished every puzzle in Advent of Code 2017!  I hope you had as much fun solving them as I had making them for you.  I'd love to hear about your adventure; you can get in touch with me via contact info on <a href="http://was.tl/" target="_blank">my website</a> or through <a href="https://twitter.com/ericwastl" target="_blank">Twitter</a> or <a href="https://hachyderm.io/@ericwastl" target="_blank">Mastodon</a>.</p>
<p class="aside">If you'd like to see more things like this in the future, please consider <a href="/2017/support" target="_blank">supporting</a> Advent of Code and sharing it with others.</p>
<p class="aside">To hear about future projects, you can follow me on <a href="https://twitter.com/ericwastl" target="_blank">Twitter</a> or <a href="https://hachyderm.io/@ericwastl" target="_blank">Mastodon</a>.</p>
<p class="aside">I've <span style="border-bottom:1px dotted #ffff66;" title="Yep, just like that.  There's at least one in the description for each day.">highlighted</span> the easter eggs in each puzzle, just in case you missed any.  Hover your mouse over them, and the easter egg will appear.</p>
<p>You can <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I+just+completed+all+25+days+of+Advent+of+Code+2017%21&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I+just+completed+all+25+days+of+Advent+of+Code+2017%21+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this moment with your friends, or <a href="/2017">[Go Check on Your Calendar]</a>.
</p></article>
<style>
.calendar-bkg {
  background: linear-gradient(to bottom, rgba(255,255,255,0) 0%,rgba(255,255,255,.3) 100%);
}
.sf {
  position: fixed;
  animation: anim-sf0 linear 20s infinite;
  z-index: -1;
}
.sf:before {
  content: "*";
}
.sf.sf0 { color: #ffffff; }
.sf.sf1 { color: #cccccc; animation-name: anim-sf1; }
.sf.sf2 { color: #999999; animation-name: anim-sf2; }
.sf.sf3 { color: #666666; animation-name: anim-sf3; }
@keyframes anim-sf0 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(8em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(8em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf1 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(6em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(6em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf2 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(4em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(4em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf3 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(2em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(2em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
</style>
<div class="sf sf0" style="top:-1.94em; left:14.61vw; animation-delay:-18.28s;"></div><div class="sf sf2" style="top:-3.92em; left:78.91vw; animation-delay:-11.32s;"></div><div class="sf sf3" style="top:-4.12em; left:47.35vw; animation-delay:-4.17s;"></div><div class="sf sf3" style="top:-4.48em; left:54.71vw; animation-delay:-7.22s;"></div><div class="sf sf1" style="top:-2.60em; left:24.67vw; animation-delay:-18.25s;"></div><div class="sf sf0" style="top:-1.44em; left:13.71vw; animation-delay:-1.73s;"></div><div class="sf sf2" style="top:-3.27em; left:61.71vw; animation-delay:-1.20s;"></div><div class="sf sf3" style="top:-4.33em; left:77.12vw; animation-delay:-7.56s;"></div><div class="sf sf0" style="top:-1.12em; left:47.53vw; animation-delay:-1.20s;"></div><div class="sf sf2" style="top:-3.82em; left:94.73vw; animation-delay:-14.02s;"></div><div class="sf sf2" style="top:-3.05em; left:53.93vw; animation-delay:-9.02s;"></div><div class="sf sf3" style="top:-4.01em; left:21.15vw; animation-delay:-15.11s;"></div><div class="sf sf2" style="top:-3.45em; left:26.89vw; animation-delay:-9.75s;"></div><div class="sf sf3" style="top:-4.58em; left:68.68vw; animation-delay:-6.29s;"></div><div class="sf sf2" style="top:-3.70em; left:5.67vw; animation-delay:-0.55s;"></div><div class="sf sf2" style="top:-3.86em; left:89.24vw; animation-delay:-5.17s;"></div><div class="sf sf3" style="top:-4.32em; left:50.08vw; animation-delay:-12.23s;"></div><div class="sf sf2" style="top:-3.68em; left:52.17vw; animation-delay:-6.04s;"></div><div class="sf sf2" style="top:-3.67em; left:83.51vw; animation-delay:-18.82s;"></div><div class="sf sf3" style="top:-4.51em; left:12.05vw; animation-delay:-0.94s;"></div><div class="sf sf2" style="top:-3.78em; left:80.91vw; animation-delay:-3.58s;"></div><div class="sf sf0" style="top:-1.41em; left:1.35vw; animation-delay:-11.33s;"></div><div class="sf sf2" style="top:-3.40em; left:68.73vw; animation-delay:-12.82s;"></div><div class="sf sf1" style="top:-2.72em; left:45.64vw; animation-delay:-13.24s;"></div><div class="sf sf2" style="top:-3.38em; left:11.34vw; animation-delay:-9.23s;"></div><div class="sf sf2" style="top:-3.17em; left:75.64vw; animation-delay:-6.26s;"></div><div class="sf sf1" style="top:-2.29em; left:32.43vw; animation-delay:-15.63s;"></div><div class="sf sf3" style="top:-4.22em; left:39.77vw; animation-delay:-12.08s;"></div><div class="sf sf2" style="top:-3.64em; left:38.23vw; animation-delay:-16.78s;"></div><div class="sf sf2" style="top:-3.36em; left:7.75vw; animation-delay:-11.86s;"></div><div class="sf sf0" style="top:-1.97em; left:89.02vw; animation-delay:-2.06s;"></div><div class="sf sf2" style="top:-3.40em; left:21.31vw; animation-delay:-3.33s;"></div><div class="sf sf3" style="top:-4.91em; left:65.32vw; animation-delay:-5.16s;"></div><div class="sf sf1" style="top:-2.48em; left:81.18vw; animation-delay:-18.30s;"></div><div class="sf sf0" style="top:-1.70em; left:30.07vw; animation-delay:-8.05s;"></div><div class="sf sf2" style="top:-3.52em; left:60.58vw; animation-delay:-9.25s;"></div><div class="sf sf3" style="top:-4.61em; left:84.23vw; animation-delay:-6.13s;"></div><div class="sf sf2" style="top:-3.17em; left:69.69vw; animation-delay:-9.55s;"></div><div class="sf sf3" style="top:-4.43em; left:44.30vw; animation-delay:-19.13s;"></div><div class="sf sf0" style="top:-1.85em; left:62.45vw; animation-delay:-16.24s;"></div><div class="sf sf2" style="top:-3.53em; left:91.94vw; animation-delay:-17.75s;"></div><div class="sf sf3" style="top:-4.64em; left:22.45vw; animation-delay:-16.36s;"></div><div class="sf sf2" style="top:-3.94em; left:48.00vw; animation-delay:-0.24s;"></div><div class="sf sf3" style="top:-4.89em; left:52.29vw; animation-delay:-18.38s;"></div><div class="sf sf1" style="top:-2.73em; left:26.19vw; animation-delay:-14.80s;"></div><div class="sf sf1" style="top:-2.90em; left:57.00vw; animation-delay:-18.75s;"></div><div class="sf sf0" style="top:-1.01em; left:19.13vw; animation-delay:-4.04s;"></div><div class="sf sf0" style="top:-1.45em; left:31.17vw; animation-delay:-10.29s;"></div><div class="sf sf3" style="top:-4.08em; left:33.83vw; animation-delay:-5.65s;"></div><div class="sf sf1" style="top:-2.67em; left:51.02vw; animation-delay:-3.87s;"></div><div class="sf sf3" style="top:-4.65em; left:47.03vw; animation-delay:-0.14s;"></div><div class="sf sf1" style="top:-2.47em; left:17.97vw; animation-delay:-7.80s;"></div><div class="sf sf3" style="top:-4.66em; left:80.80vw; animation-delay:-10.61s;"></div><div class="sf sf3" style="top:-4.88em; left:52.74vw; animation-delay:-10.35s;"></div><div class="sf sf0" style="top:-1.76em; left:68.46vw; animation-delay:-12.45s;"></div><div class="sf sf0" style="top:-1.69em; left:65.14vw; animation-delay:-19.96s;"></div><div class="sf sf3" style="top:-4.40em; left:26.54vw; animation-delay:-1.31s;"></div><div class="sf sf2" style="top:-3.11em; left:68.37vw; animation-delay:-15.09s;"></div><div class="sf sf3" style="top:-4.67em; left:44.31vw; animation-delay:-16.80s;"></div><div class="sf sf1" style="top:-2.57em; left:58.42vw; animation-delay:-0.88s;"></div><div class="sf sf0" style="top:-1.73em; left:73.83vw; animation-delay:-6.00s;"></div><div class="sf sf1" style="top:-2.42em; left:93.92vw; animation-delay:-3.56s;"></div><div class="sf sf3" style="top:-4.66em; left:25.25vw; animation-delay:-16.36s;"></div><div class="sf sf3" style="top:-4.14em; left:84.66vw; animation-delay:-14.97s;"></div><div class="sf sf2" style="top:-3.18em; left:9.49vw; animation-delay:-10.49s;"></div><div class="sf sf1" style="top:-2.42em; left:47.26vw; animation-delay:-9.57s;"></div><div class="sf sf0" style="top:-1.21em; left:53.24vw; animation-delay:-16.82s;"></div><div class="sf sf1" style="top:-2.64em; left:85.68vw; animation-delay:-14.02s;"></div><div class="sf sf0" style="top:-1.17em; left:44.35vw; animation-delay:-9.54s;"></div><div class="sf sf0" style="top:-1.52em; left:40.33vw; animation-delay:-15.59s;"></div><div class="sf sf2" style="top:-3.63em; left:59.52vw; animation-delay:-8.87s;"></div><div class="sf sf0" style="top:-1.11em; left:9.60vw; animation-delay:-9.73s;"></div><div class="sf sf2" style="top:-3.33em; left:5.37vw; animation-delay:-8.38s;"></div><div class="sf sf2" style="top:-3.73em; left:29.82vw; animation-delay:-9.54s;"></div><div class="sf sf1" style="top:-2.33em; left:38.59vw; animation-delay:-10.00s;"></div><div class="sf sf1" style="top:-2.29em; left:80.29vw; animation-delay:-10.08s;"></div><div class="sf sf2" style="top:-3.24em; left:21.06vw; animation-delay:-19.21s;"></div><div class="sf sf1" style="top:-2.30em; left:93.77vw; animation-delay:-19.21s;"></div><div class="sf sf0" style="top:-1.97em; left:41.36vw; animation-delay:-3.13s;"></div><div class="sf sf1" style="top:-2.57em; left:68.76vw; animation-delay:-13.19s;"></div><div class="sf sf1" style="top:-2.40em; left:70.27vw; animation-delay:-2.40s;"></div><div class="sf sf2" style="top:-3.43em; left:39.89vw; animation-delay:-10.13s;"></div><div class="sf sf3" style="top:-4.02em; left:29.29vw; animation-delay:-14.08s;"></div><div class="sf sf0" style="top:-1.75em; left:84.64vw; animation-delay:-17.11s;"></div><div class="sf sf2" style="top:-3.65em; left:63.22vw; animation-delay:-7.20s;"></div><div class="sf sf1" style="top:-2.42em; left:20.44vw; animation-delay:-18.16s;"></div><div class="sf sf0" style="top:-1.72em; left:35.42vw; animation-delay:-15.41s;"></div><div class="sf sf1" style="top:-2.83em; left:14.12vw; animation-delay:-9.05s;"></div><div class="sf sf1" style="top:-2.61em; left:57.24vw; animation-delay:-11.63s;"></div><div class="sf sf0" style="top:-1.21em; left:52.23vw; animation-delay:-12.52s;"></div><div class="sf sf1" style="top:-2.40em; left:56.46vw; animation-delay:-8.64s;"></div><div class="sf sf1" style="top:-2.31em; left:74.98vw; animation-delay:-2.59s;"></div><div class="sf sf2" style="top:-3.72em; left:43.64vw; animation-delay:-6.44s;"></div><div class="sf sf1" style="top:-2.24em; left:68.79vw; animation-delay:-10.53s;"></div><div class="sf sf3" style="top:-4.46em; left:74.37vw; animation-delay:-2.32s;"></div><div class="sf sf1" style="top:-2.39em; left:42.53vw; animation-delay:-16.40s;"></div><div class="sf sf3" style="top:-4.87em; left:95.63vw; animation-delay:-12.82s;"></div><div class="sf sf3" style="top:-4.87em; left:65.94vw; animation-delay:-9.97s;"></div><div class="sf sf1" style="top:-2.76em; left:59.18vw; animation-delay:-8.11s;"></div><div class="sf sf1" style="top:-2.31em; left:44.62vw; animation-delay:-12.94s;"></div>
</main>


<link href="style.css" rel="stylesheet"></link><main>
...
